# MapReduce

## ... for pedestrians

In [1]:
def square(x):
    return x**2

In [2]:
map(square, range(10))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [3]:
[x**2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [5]:
def add_numbers(x, y):
    return x + y

In [6]:
reduce(add_numbers, range(10))

45

In [7]:
sum(range(10))

45

In [8]:
add_numbers(0, add_numbers(1, add_numbers(2, add_numbers(3, 4))))

10

In [10]:
reduce(add_numbers, range(5))

10

## My first MapReduce

In [11]:
reduce(add_numbers, map(square, range(10)))

285

### but what about the keys?

In [12]:
pairs = map(lambda x: (x % 2 == 0, x**2 + x), range(10))

In [13]:
pairs

[(True, 0),
 (False, 2),
 (True, 6),
 (False, 12),
 (True, 20),
 (False, 30),
 (True, 42),
 (False, 56),
 (True, 72),
 (False, 90)]

In [14]:
from collections import defaultdict

In [18]:
def reduceByKey(reduce_fn, iterable):
    chunks = defaultdict(list)
    for k, v in iterable:
        chunks[k].append(v)
    for k in chunks:
        chunks[k] = reduce(reduce_fn, chunks[k])
    return chunks

In [19]:
reduceByKey(add_numbers, pairs)

defaultdict(list, {False: 190, True: 140})

## Staring Spark Locally

    PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS=notebook ~/src/spark-2.1.0-bin-hadoop2.7/bin/pyspark 

## The spark context

In [20]:
sc

## Baby steps

In [21]:
RDD = sc.parallelize(range(10))

In [22]:
RDD

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:475

In [23]:
RDD.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [24]:
RDD.count()

10

In [25]:
RDD.map(lambda x: x**2).collect()

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

## Loading data

In [26]:
wine = sc.textFile('data/winequality-white.csv')

In [27]:
wine.take(5)

[u'"fixed acidity";"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"pH";"sulphates";"alcohol";"quality"',
 u'7;0.27;0.36;20.7;0.045;45;170;1.001;3;0.45;8.8;6',
 u'6.3;0.3;0.34;1.6;0.049;14;132;0.994;3.3;0.49;9.5;6',
 u'8.1;0.28;0.4;6.9;0.05;30;97;0.9951;3.26;0.44;10.1;6',
 u'7.2;0.23;0.32;8.5;0.058;47;186;0.9956;3.19;0.4;9.9;6']

In [29]:
from bs4 import BeautifulSoup
import requests

In [30]:
obama_url = 'https://en.wikipedia.org/wiki/Barack_Obama'
obama_soup = BeautifulSoup(requests.get(obama_url).text, 'lxml')

for unwanted in  obama_soup(['script', 'style']):
    unwanted.decompose()
obama_text = obama_soup.get_text()

obamaRDD = sc.parallelize(obama_text.split())

## Basic RDD operations

In [37]:
obamaRDD.take(10)

[u'Barack',
 u'Obama',
 u'-',
 u'Wikipedia',
 u'Barack',
 u'Obama',
 u'From',
 u'Wikipedia,',
 u'the',
 u'free']

In [38]:
word_lengths = obamaRDD.map(lambda x: len(x))

In [39]:
word_lengths.take(5)

[6, 5, 1, 9, 6]

In [40]:
import re

In [42]:
re.match('\w+', '') == None

True

In [43]:
re.match('\w+', '-') == None

True

In [44]:
re.match('\w+', 'Obama') == None

False

In [45]:
clean_RDD = obamaRDD.filter(lambda x: re.match('\w+', x))

In [47]:
clean_RDD.take(5)

[u'Barack', u'Obama', u'Wikipedia', u'Barack', u'Obama']

In [48]:
word_lengths = clean_RDD.map(lambda x: len(x))

In [49]:
number_of_words = clean_RDD.count()

In [52]:
mean_wl = word_lengths.reduce(lambda x, y: x + y) / float(number_of_words)

In [53]:
mean_wl

5.581156112681813

In [54]:
from operator import add

In [55]:
word_lengths.reduce(add) / float(number_of_words)

5.581156112681813

In [56]:
word_lengths.sum()

163846

In [58]:
word_counts = (
    clean_RDD
      .map(lambda x: (x, 1))
      .reduceByKey(add))

In [59]:
word_counts.take(5)

[(u'essay', 1),
 (u'Sidley', 2),
 (u'Courant.', 1),
 (u'launces', 1),
 (u'reform,', 2)]

In [60]:
word_counts = (
    clean_RDD
      .map(lambda x: x.lower())
      .map(lambda x: (x, 1))
      .reduceByKey(add))

In [61]:
word_counts.take(5)

[(u'essay', 2), (u'd.', 7), (u'launces', 1), (u'hawaii.', 1), (u'reform,', 3)]

In [63]:
word_counts.takeOrdered(10, lambda (_, v): -v)

[(u'the', 1371),
 (u'of', 661),
 (u'in', 607),
 (u'retrieved', 496),
 (u'and', 493),
 (u'to', 492),
 (u'obama', 465),
 (u'on', 417),
 (u'a', 358),
 (u'from', 307)]

In [67]:
class DonTryThisAtHome(object):
    def my_add(self, a, b):
        return a + b + 1
    def sketchy_top_words(self, rdd, n):
        return (rdd
               .map(lambda x: x.lower())
               .map(lambda x: (x, 1))
               .reduceByKey(self.my_add)
               .takeOrdered(5, lambda (_, v): -v))

In [68]:
DonTryThisAtHome().sketchy_top_words(clean_RDD, 5)

[(u'the', 2741),
 (u'of', 1321),
 (u'in', 1213),
 (u'retrieved', 991),
 (u'and', 985)]

In [69]:
counter = 0
def my_count(_):
    global counter
    counter += 1

In [70]:
my_count(123122)

In [71]:
counter

1

In [72]:
obamaRDD.foreach(my_count)

In [73]:
counter

1

In [75]:
sc.parallelize(['a', 1.2, []]).collect()

['a', 1.2, []]

In [80]:
acc = sc.accumulator(0)

In [81]:
clean_RDD.foreach(lambda _: acc.add(1))

In [82]:
acc.value

29357

## Monitor Performance

http://localhost:4040/jobs/

## Set-like Operations

In [83]:
a = sc.parallelize([1,2,3,4,4])
b = sc.parallelize([3,4,5])

In [84]:
a.distinct().collect()

[1, 2, 3, 4]

In [85]:
a.union(b).collect()

[1, 2, 3, 4, 4, 3, 4, 5]

In [86]:
a.intersection(b).collect()

[3, 4]

In [88]:
a.cartesian(b).take(10)

[(1, 3),
 (1, 4),
 (1, 5),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 3),
 (3, 4),
 (3, 5),
 (4, 3)]

In [90]:
a.mean(), a.stdev(), a.sum()

(2.8, 1.16619037896906, 14)

## Other Operations of note

In [91]:
sc.parallelize(range(100)).sample(True, 0.1).collect()

[2, 3, 13, 16, 26, 42, 44, 44, 60, 64, 84]

In [92]:
a = sc.parallelize([('a', 1), ('b', 2)])
b = sc.parallelize([('c', 3), ('b', 4)])
a.join(b).collect()

[('b', (2, 4))]

In [93]:
r = sc.parallelize([(1, 1), (1, 2), (2, 1)]).groupByKey().collect()

In [94]:
r

[(1, <pyspark.resultiterable.ResultIterable at 0x10ff7ce10>),
 (2, <pyspark.resultiterable.ResultIterable at 0x10ff7c6d0>)]

In [97]:
[(k, list(v)) for k,v in r]

[(1, [1, 2]), (2, [1])]

## Aggregate

In [99]:
import numpy as np

In [100]:
def seqOp((N, partial_sum), value):
    return N + 1, partial_sum + value
def combOp((N1, partial_sum_1), (N2, partial_sum_2)):
    return N1 + N2, partial_sum_1 + partial_sum_2
values = np.random.standard_normal(1000)
N, value_sum = sc.parallelize(values).aggregate((0, 0), seqOp, combOp)

In [101]:
mean = value_sum / float(N)

In [102]:
mean

0.027450819003460306

In [103]:
values.mean()

0.02745081900346031

# Application: Search using tf-idf

In [104]:
def tf(term, RDD):
    tlower = term.lower()
    def seqOp((N, term_count), value):
        if value.lower() == tlower:
            return N+1, term_count+1
        else:
            return N+1, term_count
    def combOp((N1, c1), (N2, c2)):
        return N1+N2, c1+c2
    N, term_count = RDD.aggregate((0, 0), seqOp, combOp)
    return float(term_count) / N

In [107]:
tf('and', clean_RDD)

0.01679326906700276

In [108]:
tf('a', clean_RDD)

0.012194706543584153

In [109]:
tf('obama', clean_RDD)

0.01583949313621964

In [110]:
def makeRDD(url):
    soup = BeautifulSoup(requests.get(url).text, 'lxml')
    for unwanted in soup(['script', 'style']):
        unwanted.decompose()
    text = soup.get_text()
    return sc.parallelize(text.split()).filter(lambda x: re.match('\w+', x))

In [112]:
bushRDD = makeRDD('https://en.wikipedia.org/wiki/George_W._Bush')

In [113]:
tf('obama', bushRDD)

0.0005212858384013901

In [115]:
trumpRDD = makeRDD('https://en.wikipedia.org/wiki/Donald_Trump')

In [116]:
def search(term):
    for RDD, name in [(clean_RDD, 'obama'), (bushRDD, 'bush'), (trumpRDD, 'trump')]:
        print name, tf(term, RDD)

In [117]:
search('hotels')

obama 0.0
bush 0.0
trump 0.000260323451889


In [118]:
search('war')

obama 0.000783458800286
bush 0.00163336229366
trump 9.76212944584e-05


In [119]:
search('hawaii')

obama 0.000442824539292
bush 3.47523892268e-05
trump 6.50808629722e-05
